In [1]:
import altair as alt
import numpy as np
import pandas as pd
import random

# Blank map

In [2]:
geodata_url = 'https://gist.githubusercontent.com/jandot/ba7eff2e15a38c6f809ba5e8bd8b6977/raw/eb49ce8dd2604e558e10e15d9a3806f114744e80/belgium_municipalities_topojson.json'
geodata_feature = 'BE_municipalities'

Parameters are GeoJSON URL and feature to use.

In [3]:
def create_topo_data(url, feature):
    return alt.topo_feature(geodata_url, feature)

In [4]:
topo_municipalities = create_topo_data(geodata_url, geodata_feature)

Parameters would be GeoJSON URL, stroke color and width.

In [5]:
alt.Chart(topo_municipalities).mark_geoshape(stroke='lightgrey', strokeWidth=0.1).encode()

alt.Chart(...)

# Reading data

Parameters would be data file URL, data file type, and if necessary file encoding, column separator.

In [6]:
def read_data(uri, encoding=None, delimiter=None):
    if uri.lower().endswith('.csv'):
        return pd.read_csv(uri, encoding=encoding, delimiter=delimiter)
    elif uri.lower().endswith('.xlsx'):
        return pd.read_excel(uri)
    else:
        raise ValueError(f'file type of ""{uri}"" is unknown')

In [7]:
def validate_data(data):
    if len(data.columns) < 2:
        raise ValueError('data should have at least a NIS code and a data column')
    nis_code_pos = [idx for idx, col_name in enumerate(data.columns) if col_name.lower() == 'niscode']
    if len(nis_code_pos) > 1:
        raise ValueError('multiple columns have NIS code name')
    elif len(nis_code_pos) == 0:
        raise ValueError('NIS code column is missing')
    else:
        columns = list(data.columns)
        columns[nis_code_pos[0]] = 'niscode'
        data.columns = columns
    if data.niscode.dtype != np.int64:
        raise ValueError('data type for NIS code is incorret, should be integer')

In [8]:
data = read_data('data/Gemeenten-latest.csv', encoding='iso-8859-1', delimiter=';')

In [9]:
validate_data(data)

In [10]:
data.head()

,ID,Gemeente,Bevolking,niscode,DATE,COUNT
0,1,AARTSELAAR,14423,11001,2020-03-31,0
1,2,ANTWERPEN,527763,11002,2020-03-31,52
2,3,BOECHOUT,13373,11004,2020-03-31,0
3,4,BOOM,18526,11005,2020-03-31,1
4,5,BORSBEEK,10943,11007,2020-03-31,0


Select only data from March 31th, 2020.

In [14]:
data = data.query('DATE == "2020-03-31"').copy()

In [15]:
data.head()

,ID,Gemeente,Bevolking,niscode,DATE,COUNT,color
0,1,AARTSELAAR,14423,11001,2020-03-31,0,green
1,2,ANTWERPEN,527763,11002,2020-03-31,52,yellow
2,3,BOECHOUT,13373,11004,2020-03-31,0,red
3,4,BOOM,18526,11005,2020-03-31,1,blue
4,5,BORSBEEK,10943,11007,2020-03-31,0,red


Add a column with random qualitative data.

In [16]:
data['color'] = random.choices(['red', 'green', 'blue', 'yellow'], k=len(data))

In [17]:
data.head()

,ID,Gemeente,Bevolking,niscode,DATE,COUNT,color
0,1,AARTSELAAR,14423,11001,2020-03-31,0,green
1,2,ANTWERPEN,527763,11002,2020-03-31,52,blue
2,3,BOECHOUT,13373,11004,2020-03-31,0,yellow
3,4,BOOM,18526,11005,2020-03-31,1,yellow
4,5,BORSBEEK,10943,11007,2020-03-31,0,red


In [ ]:
data.to_csv('data/data.csv')

# Putting data on the map

## Quantivative data

Parameters would be legend title, color scheme, data domain (min. value, max. value).

In [27]:
def guess_data_type(data, columns):
    types = dict()
    for column in columns:
        pandas_type = data[column].dtype
        if pandas_type in [np.int32, np.int64, np.float32, np.float64]:
            types[column] = 'Q'
        else:
            types[column] = 'N'
    return types

In [37]:
def create_plot(topo_data, data, column_name, data_type, tooltips = None,
                stroke='lightgrey', strokeWidth=0.5, legend_title=None,
                schema='reds'):
    if legend_title is None:
        legend_title = column_name
    if tooltips is None:
        lookup = [column_name]
    else:
        lookup = [column_name] + list(tooltips)
    print(lookup)
    return alt.Chart(topo_data).mark_geoshape(stroke=stroke, strokeWidth=strokeWidth).encode(
                color=alt.Color(f'{column_name}:{data_type}',
                                legend=alt.Legend(title=legend_title), 
                                scale=alt.Scale(scheme=schema)),
                tooltip=[f'{cname}:N' for cname in lookup],
        ).transform_lookup(
            lookup='properties.CODE_INS',
            from_=alt.LookupData(data, 'niscode', lookup)
        )

In [38]:
def create_quantitative_plot(topo_data, data, column_name, tooltips=None,
                             stroke='lightgrey', strokeWidth=0.5, legend_title=None,
                             schema='reds'):
    return create_plot(topo_data, data, column_name, 'Q',
                tooltips, stroke, strokeWidth, legend_title, schema)

In [39]:
create_quantitative_plot(topo_data=topo_municipalities, data=data, column_name='Bevolking',
                         tooltips=['Gemeente', 'COUNT'])

['Bevolking', 'Gemeente', 'COUNT']


alt.Chart(...)

## Nominal data

Parameters would be legend title, color scheme.

In [ ]:
def create_nominal_plot(topo_data, data, column_name,
                             stroke='lightgrey', strokeWidth=0.5, legend_title=None,
                             schema='reds'):
    return create_plot(topo_data, data, column_name, 'N',
                stroke, strokeWidth, legend_title, schema)

In [ ]:
create_nominal_plot(topo_data=topo_municipalities, data=data.query('DATE == "2020-03-31"'),
                    column_name='color', legend_title='Kleur', schema='category10')